In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
from pyspark.sql.functions import expr, array, col, explode, arrays_zip, lit, avg, when, broadcast

# warehouse_location points to the default location for managed databases and tables
warehouse = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
jobs_df = spark.table("database.jobs")
popu_df = spark.table("database.population")
jobs_df = jobs_df.filter(col("indicador_name").like("%Labor force with advanced education%"))
jobs_df = jobs_df.drop(col("pais_cod"), col("indicador_code"))
popu_df = popu_df.filter(col("cod_indicador").like("%SP.POP.TOTL%"))
popu_df = popu_df.drop(col("cod_pais"), col("cod_indicador"), col("nome_indicador"), col("Country_name"), col("Country_code"), col("Indicator_name"), col("Indicator_code"))
popu_df = popu_df.withColumnRenamed("valor", "valorI")
popu_df = popu_df.withColumnRenamed("nome_pais", "pais_name")
popu_df = popu_df.withColumn("ano", col("ano").cast(IntegerType()))

In [3]:
paises = ["%Liechtenstein%", "%Germany%", "%Denmark%", "%Switzerland%", "%Poland%", "%Czech%", 
          "%Slovakia%", "%Slovenia%", "%Hungary%", "%Austria%"]
anos = ["2010", "2011", "2012", "2013", "2014", "2015"]

jobs_df = jobs_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2]) |
    col("pais_name").like(paises[3]) |
    col("pais_name").like(paises[4]) |
    col("pais_name").like(paises[5]) |
    col("pais_name").like(paises[6]) |
    col("pais_name").like(paises[7]) |
    col("pais_name").like(paises[8]) |
    col("pais_name").like(paises[9])
).filter(
    col("ano").isin(anos) &
    col("valor").isNotNull()
)

popu_df = popu_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2]) |
    col("pais_name").like(paises[3]) |
    col("pais_name").like(paises[4]) |
    col("pais_name").like(paises[5]) |
    col("pais_name").like(paises[6]) |
    col("pais_name").like(paises[7]) |
    col("pais_name").like(paises[8]) |
    col("pais_name").like(paises[9])
).filter(
    col("ano").isin(anos) &
    col("valorI").isNotNull()
)

In [4]:
literacia_df = popu_df.join(jobs_df, ["pais_name", "ano"], "left")
literacia_df = literacia_df.withColumn("valorI", ((col("valor") / 100) * (col("valorI") / 2)).cast("int"))
literacia_df = literacia_df.withColumnRenamed("pais_name", "paises")

In [5]:
literacia_df = literacia_df.withColumn("genero",
    when(col("indicador_name").like("% female%"), "F")
    .when(col("indicador_name").like("% male%"), "M")
)

literacia_df = literacia_df.filter(col("genero").isNotNull())

literacia_df = literacia_df.drop(col("indicador_name"))

In [6]:
literacia_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Projeto/gold/TabelaLaborA/")

In [7]:
spark.sql(
    """
    SELECT * FROM database.LaborAGold
    """
).show()

+-------+----+-------+--------+------+
| paises| ano|  valor|  valorI|genero|
+-------+----+-------+--------+------+
|Denmark|2010|80.1306| 2222695|     F|
|Denmark|2011|79.4572| 2213110|     F|
|Denmark|2012|79.5007| 2222669|     F|
|Denmark|2013|78.3737| 2200315|     F|
|Denmark|2014|78.1326| 2204696|     F|
|Denmark|2015|78.4618| 2229681|     F|
|Denmark|2010|78.9792| 2190757|     M|
|Denmark|2011|79.1329| 2204077|     M|
|Denmark|2012|78.9636| 2207653|     M|
|Denmark|2013|78.2359| 2196446|     M|
|Denmark|2014|77.0424| 2173934|     M|
|Denmark|2015|77.1271| 2191752|     M|
|Germany|2010|72.9555|29830382|     F|
|Germany|2011|73.8338|29635036|     F|
|Germany|2012|73.6189|29604302|     F|
|Germany|2013|73.4874|29632178|     F|
|Germany|2014|73.1157|29605459|     F|
|Germany|2015|72.8219|29742869|     F|
|Germany|2010|73.4659|30039076|     M|
|Germany|2011|74.3946|29860126|     M|
+-------+----+-------+--------+------+
only showing top 20 rows



In [8]:
spark.stop()